In [1]:
!pip uninstall -y torch
!pip install torch==1.8.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install --no-index --no-cache-dir torch-scatter  torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.1+cu101.html
!pip install --no-cache-dir torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.1+cu101.html
!pip install --no-cache-dir torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.1+cu101.html
!pip install --no-cache-dir torch-geometric
!pip install wandb

Found existing installation: torch 1.6.0+cu101
Uninstalling torch-1.6.0+cu101:
  Successfully uninstalled torch-1.6.0+cu101
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu101/torch-1.8.1%2Bcu101-cp37-cp37m-linux_x86_64.whl (763.7 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.7.0+cu101 requires torch==1.6.0, but you have torch 1.8.1+cu101 which is incompatible.
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.1+cu101.html
     |████████████████████████████████| 7.9 MB 6.1 MB/s            
     |████████████████████████████████| 3.1 MB 328 kB/s            
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.1+cu101.html
     |████████████████████████████████| 1.5 MB 607 kB/s            
Looking in links: https://pytorch-geometric.com/whl/torch-1.8.1+

In [16]:
from tqdm import tqdm
from collections import Counter
import random
    
import pandas as pd
import numpy as np

import networkx as nx
import csrgraph as cg

from gensim.models import Word2Vec

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import ModuleList, Embedding
from torch.autograd import Variable

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import PNAConv, BatchNorm, global_add_pool
from torch_geometric.data import Data
from torch_geometric.utils import negative_sampling
from torch_geometric.utils import degree
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.data import HeteroData

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

General data:

In [2]:
authors_edges_papers_general = pd.read_csv("../processed_data/SSORC_CS_2010_2021_authors_edges_papers_indices.csv", index_col = 0, \
                                   converters={"papers_indices": lambda x: x.strip("[]").replace("'","").split(", ")})
authors_edges_general = pd.read_csv("../processed_data/SSORC_CS_2010_2021_authors_edge_list.csv", index_col = 0)

/home/user/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
papers_features_general = pd.read_csv("../processed_data/SSORC_CS_2010_2021_papers_features_vectorized_compressed_32.csv", index_col = 0)
authors_features_general = pd.read_csv("../processed_data/SSORC_CS_2010_2021_authors_features.csv", index_col = 0)

In [4]:
aev = authors_edges_general.values
edge_to_index = {(aev[i][0], aev[i][1]):i for i in tqdm(range(len(aev)))}
index_to_edge = {i:(aev[i][0], aev[i][1]) for i in tqdm(range(len(aev)))}

100%|██████████| 30796749/30796749 [00:26<00:00, 1178220.82it/s]


Local data:

In [5]:
dataset_name = 'SSORC_CS_10_21_419_2055_primus' #'SSORC_CS_10_21_3340_16830_115907_primus'

In [6]:
authors_edges_papers = pd.read_csv("../datasets/" + dataset_name + "/" + dataset_name + "_" + "authors_edges_papers_indices.csv", index_col = 0,\
                                   converters={"papers_indices": lambda x: x.strip("[]").replace("'","").split(", ")})

In [7]:
authors_graph = nx.read_edgelist("../datasets/" + dataset_name + "/" + dataset_name + "_" + "authors.edgelist", create_using = nx.DiGraph)

In [8]:
citation_graph = nx.read_edgelist("../datasets/" + dataset_name + "/" + dataset_name + "_" + "papers.edgelist", create_using = nx.DiGraph)

### MANDATORY CHECK

In [9]:
sAe = list(authors_graph.edges)
sAe = [(int(sAe[i][0]), int(sAe[i][1])) for i in range(len(sAe))]

In [10]:
authors_edges_papers_sub_2 = [authors_edges_papers["papers_indices"][edge_to_index[sAe[i]]] for i in tqdm(range(len(sAe)))]
authors_edges_papers_sub_2_edges = [sAe[i] for i in tqdm(range(len(sAe)))]
authors_edges_papers_sub_flat_2 = [int(item) for subarray in authors_edges_papers_sub_2 for item in subarray]
unique_papers_2 = list(set(authors_edges_papers_sub_flat_2))

100%|██████████| 1753/1753 [00:00<00:00, 3478058.14it/s]


In [11]:
cgn = list(citation_graph.nodes())
cgn = [int(cgn[i]) for i in range(len(cgn))]

In [12]:
len(set(unique_papers_2).intersection(set(cgn))), len(unique_papers_2), len(cgn)

(1806, 1806, 2055)

In [ ]:
assert len(set(unique_papers_2).intersection(set(cgn))) == len(unique_papers_2)

### Model

In [167]:
a_graph, c_graph = cg.csrgraph(authors_graph, threads = 96), cg.csrgraph(citation_graph, threads = 96) 
a_node_names, c_node_names = a_graph.names, c_graph.names

walks = a_graph.random_walks(walklen=3, # length of the walks
                epochs=1, # howm any times to start a walk from each node
                start_nodes=None, # the starting node. It is either a list (e.g., [2,3]) or None. If None it does it on all nodes and returns epochs*G.number_of_nodes() walks
                return_weight=1.,
                neighbor_weight=1.)

In [275]:
len(walks)

419

In [168]:
papers_to_edges = dict()
for paper in unique_papers_2:
    paper = str(paper)
    for i in range(len(authors_edges_papers_sub_2)):
        if str(paper) in authors_edges_papers_sub_2[i]:
            if paper in papers_to_edges:
                papers_to_edges[paper].append(sAe[i])
            else:
                papers_to_edges[paper] = [sAe[i]]

In [169]:
c_node_names_reverse = {c_node_names[i]:i for i in range(len(c_node_names))}
a_node_names_reverse = {a_node_names[i]:i for i in range(len(a_node_names))}

In [203]:
authors_graph_und = authors_graph.to_undirected()

In [273]:
import random

extensions = []
for i in tqdm(range(len(walks))):
    adj_edges = list(authors_graph_und.edges(node_names[walks[i][0]]))
    adj_edges = [(int(adj_edges[h][0]), int(adj_edges[h][1])) for h in range(len(adj_edges))]
    adj_edges_corrected = []
    for h in range(len(adj_edges)):
        if adj_edges[h] not in sAe:
            adj_edges[h] = (adj_edges[h][1], adj_edges[h][0])
    edge_papers = [authors_edges_papers["papers_indices"][edge_to_index[adj_edges[g]]] for g in range(len(adj_edges))]
    edge_papers = random.choice(edge_papers)
#    print(edge_papers)
    addition = []
    for paper in edge_papers:
        start_node_c = c_node_names_reverse[paper]
        walks_c = c_graph.random_walks(walklen=6,
                    epochs=2,
                    start_nodes=[start_node_c],
                    return_weight=1.,
                    neighbor_weight=1.)
        terminus_cited_papers = list(set(np.array(walks_c)[:, -1]))
        try:
            a_edges = [papers_to_edges[str(c_node_names[terminus_cited_papers[h]])] for h in range(len(terminus_cited_papers))]
        except:
            print("Missing")
            continue 
        a_edges = [node for edge in a_edges for node in edge]
        a_nodes = [node for edge in a_edges for node in edge]
        a_nodes_local = [a_node_names_reverse[str(a_nodes[i])] for i in range(len(a_nodes))] 
        addition += a_nodes_local
    extensions.append(addition[0:10])


100%|██████████| 419/419 [00:00<00:00, 1323.64it/s]

Missing
Missing
Missing
Missing
Missing
Missing


In [279]:
walks = walks.tolist()
extended_walks = []
for i in range(len(walks)):
    extended_walks.append(walks[i] + extensions[i])

In [36]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        print('Loss after epoch {}: {}'.format(self.epoch, loss))
        self.epoch += 1

In [298]:
model = Word2Vec(walks, vector_size=128, window=5, min_count=1, epochs = 10000,
                 compute_loss=True, callbacks=[callback()])

Loss after epoch 0: 4355.6015625
Loss after epoch 1: 8652.6396484375
Loss after epoch 2: 12929.7978515625
Loss after epoch 3: 17187.2265625
Loss after epoch 4: 21490.15234375
Loss after epoch 5: 25800.82421875
Loss after epoch 6: 30110.875
Loss after epoch 7: 34483.0625
Loss after epoch 8: 38869.3359375
Loss after epoch 9: 43182.9765625
Loss after epoch 10: 47477.64453125
Loss after epoch 11: 51836.67578125
Loss after epoch 12: 56241.41015625
Loss after epoch 13: 60630.3046875
Loss after epoch 14: 65002.95703125
Loss after epoch 15: 69284.421875
Loss after epoch 16: 73545.828125
Loss after epoch 17: 77832.4296875
Loss after epoch 18: 82198.484375
Loss after epoch 19: 86582.8125
Loss after epoch 20: 90888.375
Loss after epoch 21: 95174.078125
Loss after epoch 22: 99519.7109375
Loss after epoch 23: 103890.7265625
Loss after epoch 24: 108175.65625
Loss after epoch 25: 112447.53125
Loss after epoch 26: 116676.71875
Loss after epoch 27: 120830.0234375
Loss after epoch 28: 124974.7734375
Los

In [299]:
author_vectors = model.wv

In [300]:
embeddings = []
for i in range(len(author_vectors)):
    embeddings.append(author_vectors[i])

In [302]:
pd.DataFrame(np.array(embeddings), index = list(a_node_names)).to_csv("makarov_embeddings_1.csv")